# Notebook to reproduce the MLTA case study results

The results were created using Win10, an AMD RYZEN 5 3600XT with 32GB RAM (4 x 8GB of Crucial Ballistix BL2K8G36C16U4B 3600 MHz with CL16 timings (i.e. overclocked to 3600Mhz in the bios)), and a fast NMVE SSD (Samsung 980 M.2 NVME 1TB SSD) on a PCIE 4.0 slot with up to 3 GB/s read/write speed. Other hardware may yield different results because all analysis are time-bound.

If you fail to reproduce any of the results please open a Github issue, if this is left unresponded you can contact me via ljpcvg@gmail.com 

## Time-restricted analysis


We use the metadataset for the meta-learners to learn from and to create their result, which can be accessed via the readme. Note that you could also use another metadataset containing information on OpenML18CC datasets to get the results, but these may vary more from what we presented. We assume we already pre-computed characterizations and stored them in the dataset, which can be done as in code example 1 in the readme.

Each cell contains the code to get the results for one of the meta-learners. Running the cell will vary in length, we have ran them in batches, which is possible via the `dataset_ids` argument in `evaluate()`. Depending on the meta-learning approach each cell may require half a day up to a few days in run time.

In [2]:
import os

from metadatabase import MetaDataBase
from evaluation import LeaveOneDatasetOutEvaluation
from metalearners import TopSimilarityLearner, TopXGBoostRanked, UtilityEstimateLearner, AverageRegretLearner, PortfolioBuilding
from dataset_characterization import WistubaMetaFeatures, FeurerMetaFeatures, PreTrainedDataset2Vec
from configuration_characterization import AlgorithmsPropositionalization, RankMLPipelineRepresentation
from dataset_similarity import CharacterizationSimilarity
from gama.configuration.classification import clf_config

### BEWARE: <br/>
RUNNING THESE CELLS WILL PRODUCE .CSV FILES WITH THE SAME NAMES AS ALREADY PRESENT IN THE ROOT DIR EVALUATION_RESULTS, IT WILL THUS OVERWRITE ALREADY PRESENT RESULTS, CHANGE THE PATH IF YOU WANT TO KEEP BOTH.

______

Average Regret

In [ ]:
meta_learner_name = "average_regret"
online_phase_time = 300  # in seconds
max_eval_time = 600 # max evaluation time per configuration, thus one dataset may still take 4+ hours, of which we have 72
jobs = 1  # n_jobs used in the online phase, typically not the intensive part, the offline phase is
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = AverageRegretLearner()

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                max_time=max_eval_time)

# store the evaluation results
store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

________

Portfolio Building

In [ ]:
meta_learner_name = "portfolio_building"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
# load and use pre-computed results matrix, otherwise need to run a 1+ day offline phase 72 times, would be infeasible.
dataset_characterizations = mdbase.get_dataset_characterizations("portfolio_results_matrix")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = PortfolioBuilding()

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_____

UtilityEstimate with Wistuba meta-features

In [ ]:
meta_learner_name = "utility_estimate_wistuba"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("wistuba_metafeatures")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = UtilityEstimateLearner(CharacterizationSimilarity(WistubaMetaFeatures()))

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

________

UtilityEstimate with Feurer meta-features

In [ ]:
meta_learner_name = "utility_estimate_feurer"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("wistuba_metafeatures")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = UtilityEstimateLearner(CharacterizationSimilarity(FeurerMetaFeatures()))


evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

____________

UtilityEstimate with Dataset2Vec characterization

In [ ]:
meta_learner_name = "utility_estimate_d2v"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("dataset2vec_split0_10batches")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = UtilityEstimateLearner(CharacterizationSimilarity(PreTrainedDataset2Vec()))

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_________

TopSimilarity with Wistuba meta-features

In [ ]:
meta_learner_name = "topsim_wistuba"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("wistuba_metafeatures")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopSimilarityLearner(CharacterizationSimilarity(WistubaMetaFeatures()))

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

________

TopSimilarity with Feurer meta-features

In [ ]:
meta_learner_name = "topsim_feurer"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("feurer_metafeatures")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopSimilarityLearner(CharacterizationSimilarity(FeurerMetaFeatures()))

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_______

TopSimilarity with Dataset2Vec characterization

In [ ]:
meta_learner_name = "topsim_d2v"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("dataset2vec_split0_10batches")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopSimilarityLearner(CharacterizationSimilarity(PreTrainedDataset2Vec()))

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                max_time=max_eval_time)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_________

RankML with Wistuba meta-features

In [ ]:
meta_learner_name = "xgboostranker_rankml_wistuba"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("wistuba_metafeatures")
config_characterizations = mdbase.get_configuration_characterizations("rankml_pipeline_representation")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopXGBoostRanked(WistubaMetaFeatures(), RankMLPipelineRepresentation(mdbase))
metalearner_kwargs = {"dataset_characterizations_name": "wistuba_metafeatures",
                      "configuration_characterizations_name": "rankml_pipeline_representation",
                      "max_n_models": 500}  # online phase

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                config_characterizations=config_characterizations,
                                                max_time=max_eval_time,
                                                metalearner_kwargs=metalearner_kwargs)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_________

RankML with Feurer meta-features

In [ ]:
meta_learner_name = "xgboostranker_rankml_feurer"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("feurer_metafeatures")
config_characterizations = mdbase.get_configuration_characterizations("rankml_pipeline_representation")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopXGBoostRanked(FeurerMetaFeatures(), RankMLPipelineRepresentation(mdbase))
metalearner_kwargs = {"dataset_characterizations_name": "feurer_metafeatures",
                      "configuration_characterizations_name": "rankml_pipeline_representation",
                      "max_n_models": 500}  # online phase

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                config_characterizations=config_characterizations,
                                                max_time=max_eval_time,
                                                metalearner_kwargs=metalearner_kwargs)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

______

RankML with Dataset2Vec characterizations

In [ ]:
meta_learner_name = "xgboostranker_rankml_d2v"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("dataset2vec_split0_10batches")
config_characterizations = mdbase.get_configuration_characterizations("rankml_pipeline_representation")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopXGBoostRanked(PreTrainedDataset2Vec(), RankMLPipelineRepresentation(mdbase))
metalearner_kwargs = {"dataset_characterizations_name": "dataset2vec_split0_10batches",
                      "configuration_characterizations_name": "rankml_pipeline_representation",
                      "max_n_models": 500}  # online phase

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                config_characterizations=config_characterizations,
                                                max_time=max_eval_time,
                                                metalearner_kwargs=metalearner_kwargs)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_________

Algorithms Propositionalization with Wistuba meta-features

In [ ]:
meta_learner_name = "xgboostranker_ap_wistuba"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("wistuba_metafeatures")
config_characterizations = mdbase.get_configuration_characterizations("algorithms_propositionalization")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopXGBoostRanked(WistubaMetaFeatures(), AlgorithmsPropositionalization(clf_config))
metalearner_kwargs = {"dataset_characterizations_name": "wistuba_metafeatures",
                      "configuration_characterizations_name": "algorithms_propositionalization",
                      "max_n_models": 500}  # online phase

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                config_characterizations=config_characterizations,
                                                max_time=max_eval_time,
                                                metalearner_kwargs=metalearner_kwargs)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_____

Algorithms Propositionalization with Feurer meta-features

In [ ]:
meta_learner_name = "xgboostranker_ap_feurer"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("feurer_metafeatures")
config_characterizations = mdbase.get_configuration_characterizations("algorithms_propositionalization")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopXGBoostRanked(FeurerMetaFeatures(), AlgorithmsPropositionalization(clf_config))
metalearner_kwargs = {"dataset_characterizations_name": "feurer_metafeatures",
                      "configuration_characterizations_name": "algorithms_propositionalization",
                      "max_n_models": 500}  # online phase

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                config_characterizations=config_characterizations,
                                                max_time=max_eval_time,
                                                metalearner_kwargs=metalearner_kwargs)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

_______

Algorithms Propositionalization with Dataset2Vec characterizations

In [ ]:
meta_learner_name = "xgboostranker_ap_d2v"
online_phase_time = 300  # in seconds
max_eval_time = 600
jobs = 1
file_name = f"{meta_learner_name}_{online_phase_time}s.csv"

mdbase = MetaDataBase("metadatabase_openml18cc")
dataset_characterizations = mdbase.get_dataset_characterizations("dataset2vec_split0_10batches")
config_characterizations = mdbase.get_configuration_characterizations("algorithms_propositionalization")

evaluation_method = LeaveOneDatasetOutEvaluation(max_time=online_phase_time, n_jobs=jobs)
metalearner = TopXGBoostRanked(PreTrainedDataset2Vec(), AlgorithmsPropositionalization(clf_config))
metalearner_kwargs = {"dataset_characterizations_name": "dataset2vec_split0_10batches",
                      "configuration_characterizations_name": "algorithms_propositionalization",
                      "max_n_models": 500}  # online phase

evaluation_results = evaluation_method.evaluate(mdbase,
                                                metalearner, 
                                                dataset_characterizations=dataset_characterizations, 
                                                config_characterizations=config_characterizations,
                                                max_time=max_eval_time,
                                                metalearner_kwargs=metalearner_kwargs)

store_path = os.path.join("evaluation_results", file_name)
evaluation_method.store_results(store_path, meta_learner_name)

________

## Warm-starting analysis


Code to reproduce the portfolio building warm-starting results. Note that we ran the evaluation in batches, and we advise you to do the same. Each dataset takes at least 3 hours without re-training and evaluating the best model. Hence a batch of 4 datasets will take 12 hours to run.

BEWARE: the full warm-starting analysis will take 216+ hours using 32GB of RAM and 8 threads.

In [3]:
from sklearn.model_selection import StratifiedKFold
from utilities import warm_started_gama
from metadatabase import MetaDataBase
from metalearners import PortfolioBuilding
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from utilities.gama import get_fixed_preprocessing_steps
import numpy as np
import pandas as pd
from datetime import datetime

BEWARE: CHANGE BATCH NUMBER

In [ ]:
# run in batches, DONT FORGET TO SELECT BATCH
batch_1 = [0, 1, 2, 3, 4, 5]  
batch_2 = [6, 7, 8]          
batch_3 = [9, 10, 11]         
batch_4 = [12, 13, 14, 15]    
batch_5 = [16, 17, 18, 19]    
batch_6 = [20, 21, 22, 23]    
batch_7 = [24, 25, 26, 27]    
batch_8 = [29, 30, 31]      
batch_9 = [33, 34, 35]          
batch_10 = [36, 37, 38, 39, 71]
batch_11 = [40, 41, 42, 43]     
batch_12 = [44, 45, 46, 47]   
batch_13 = [48, 49, 50]        
batch_14 = [52, 53, 54, 55]    
batch_15 = [56, 57, 58, 59]     
batch_16 = [60, 61, 62, 63]   
batch_17 = [64, 65, 66, 67]   
batch_18 = [68, 69, 70]         
batch_19 = [28, 32]            
batch_20 = [51]                 

dataset_ids = batch_1 # TODO TODO TODO ADAPT BATCH EACH RUN TODO TODO TODO

In [ ]:
approach_name = "portfolio_building_warm_start_asyncea"
online_phase_time = 300  # in seconds
max_gama_time = 3600
gama_jobs = 8
n_splits = 3

mdbase = MetaDataBase("metadatabase_openml18cc")
metalearner = PortfolioBuilding()

In [ ]:
# GAMA with K-Fold
cv_avg = {}
for id in dataset_ids:
    X, y = mdbase.get_dataset(id)
    evaluation_scores = []
    # 3 fold CV warm-started GAMA run.
    for i, (train_index, test_index) in enumerate(StratifiedKFold(n_splits=n_splits).split(X, y)):
        X_train = X.iloc[train_index]
        y_train = pd.DataFrame(y.iloc[train_index])
        X_test = X.iloc[test_index]
        y_test = pd.DataFrame(y.iloc[test_index])

        datasets_to_keep = mdbase.list_datasets(by="id")
        datasets_to_keep.remove(id)
        mdbase.partial_datasets_view(datasets_to_keep)  # Do not pass to-be-learned dataset's information to learners offline phase.
        results_matrix = mdbase.get_dataset_characterizations("portfolio_results_matrix", include_dimension_names=True)
        metalearner.offline_phase(mdbase=mdbase, results_matrix=results_matrix)
        
        # It could be GAMA does not evaluate any individuals therefore try warm-start, otherwise exclude the run from score computation
        try:
            gama = warm_started_gama(metalearner, X_train, y_train, online_phase_max_time=online_phase_time, 
                                    max_total_time=max_gama_time, n_jobs=gama_jobs, logs_path="gama_logs/_d{}_f{}".format(id, i),
                                    max_memory_mb=32768)
            # get gama's best-evaluated sklearn pipeline
            partial_pipe = gama._evaluation_library.n_best(n=1, with_pipelines=True)[0].individual.pipeline
            preprocessing_steps = get_fixed_preprocessing_steps(X_train)
            pipe_steps = []
            for key in partial_pipe.named_steps:
                if key != "imputation":
                    pipe_steps.append((key, partial_pipe.named_steps[key]))
            full_best_pipe = Pipeline(preprocessing_steps + pipe_steps)
            full_best_pipe.fit(X_train, y_train)

            # get the test score, no need to encapsulate with try/except because it was already succesfully fitted in GAMA.
            y_pred = full_best_pipe.predict_proba(X_test)
            labels = np.unique(LabelEncoder().fit_transform(y))
            performance = float(-1 * log_loss(y_true=LabelEncoder().fit_transform(y_test), y_pred=y_pred, labels=labels))
            evaluation_scores.append(performance)
            mdbase.restore_view()
            metalearner.clear_configurations()
            print(f"done with fold: {i}, with a score of: {performance}")

        except ValueError as e:
            print("caught ValueError: ", repr(e))
            if str(e) == "population must be at least size 3 for a pair to be selected":   
                print(f"Had too little individuals in the population for this fold {i}, skip it.")
                mdbase.restore_view()
                continue
            else:  # only catch the specific ValueError for not enough population, otherwise raise it.
                raise e
        
    
    cv_avg[id] = np.mean(evaluation_scores)
    print(f"Dataset with id: {id} attained a score of {cv_avg[id]} at {str(datetime.now())}")

# always write the results to a .csv file with the dataset ids
results = []
file_name = "results_" + approach_name
for id in dataset_ids:
    file_name += f"_d{id}"
    score = cv_avg[id]
    results.append([f"{approach_name}", id, score])

warm_start_results_df = pd.DataFrame(np.array(results), columns=['name', 'dataset_name', 'neg_log_loss'])
warm_start_results_df.to_csv(file_name + ".csv", index=False)